In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import librosa
import pandas as pd

# Another version


In [2]:
import json
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoModel, AutoTokenizer, get_scheduler ,AutoModelForMaskedLM
import re
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW,Adam
from tqdm.notebook import tqdm, trange
from time import perf_counter
from PIL import Image
import pandas as pd

In [3]:
# set random seeds for repeatability
import numpy as np
import random

def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
seed_val = 0
set_seed(seed_val)

In [4]:
import pandas as pd
df=pd.read_csv('/kaggle/input/dsc-round1/train_ocr.csv')
len(df)

10805

In [5]:
import json
import pandas as pd
with open('/kaggle/input/dsc2024/vimmsd-private-test.json', 'r') as f:
    data = json.load(f)

# Sử dụng json_normalize để làm phẳng dữ liệu trong "root"
df_test = pd.DataFrame.from_dict(data, orient='index')
df_test.sample()

,image,caption,label
1445,4d0a9e0270200b2fd565c3a10446f08f96149d87ac901b...,"Thiếu t.iền cũng được, đừng thiếu lòng tự trọn...",None


0


In [6]:
df['label'].value_counts()

label
not-sarcasm      6062
multi-sarcasm    4224
image-sarcasm     442
text-sarcasm       77
Name: count, dtype: int64

In [7]:
# Create a copy of df for split_1
split_1 = df.copy()

# Create the 'not-sarcasm' column in the copy
# Modify the 'label' column in split_1
split_1['label'] = split_1['label'].apply(lambda x: 'not-sarcasm' if x == 'not-sarcasm' else 'sarcasm')

# Display the first few rows to verify
split_1.sample(10)
# Display the first few rows to verify
len(split_1)


10805

In [8]:
# Filter the DataFrame for the specified labels and create a copy
split_2 = df[df['label'].isin(['image-sarcasm', 'text-sarcasm', 'multi-sarcasm'])].copy()

# Create a new column 'multi-sarcasm' in split_2
split_2['label'] = split_2['label'].apply(lambda x: 'multi-sarcasm' if x == 'multi-sarcasm' else 'not-multi-sarcasm')

# Display the first few rows to verify the new column
split_2.sample(5)
len(split_2)

4743

In [9]:
# Filter the DataFrame for 'image-sarcasm' and 'text-sarcasm' and create a copy
split_3 = df[df['label'].isin(['image-sarcasm', 'text-sarcasm'])].copy()

# Encode 'image-sarcasm' as 1 and 'text-sarcasm' as 0
split_3['label'] = split_3['label'].apply(lambda x: 'image-sarcasm' if x == 'image-sarcasm' else 'not-image-sarcasm')

# Display the first few rows to verify
(split_3.sample(5))
len(split_3)

519

In [10]:
type = 'gfdgfdg'
# Image processing condition
if type == 'not':
    df_split = split_1.copy()
elif type == 'multi':
    df_split = split_2.copy()
elif type == 'image':
    df_split = split_3.copy()
else:
    df_split = df.copy()

# Fill NaNs with 'sarcasm'
df_split = df_split.applymap(lambda x: 'sarcasm' if x == ' ' else x)

# Display a random sample of 20 rows
df_split.sample(10)


/tmp/ipykernel_30/548335595.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_split = df_split.applymap(lambda x: 'sarcasm' if x == ' ' else x)


,Unnamed: 0,_key,image,caption,label,ocr1
9424,9424,9424,b93ba0882a054668e2f41d23bc5fe08c220c01571edd2c...,Làm gì cũng phải có VĂN!,multi-sarcasm,"""Nó hành tao tốn tiền quán"", ""bán mẹ đi cho kh..."
1560,1560,1560,6a0c5567b403058457571021b1140a8f8395298fc3b679...,Tạm quên đi sự thất vọng và tiếc nuối sau trận...,not-sarcasm,"Anh Vria, 16, ĐTU22 Việt Nam, thì tích cực tập..."
9190,9190,9190,7f48657379fec679fd331f673639267455e5b104ed4c6b...,Xiumin (EXO) đã hạ cánh tại Sân bay nội bài tr...,not-sarcasm,09
6775,6775,6775,2a50365927daa8d28c0d4f8d83ad500918f286a839a6fd...,THUỲ TIÊN ĐẸP KHÔNG GÓC CHẾT BÊN CẠNH YAMAHA G...,not-sarcasm,sarcasm
2808,2808,2808,a91f0b7c22a49dd791417e69bb724f3186d656f5a37c39...,Hợp lý,multi-sarcasm,"Làm thế nào để hô biến mức lương 5trung, 1 thà..."
7031,7031,7031,8d9536dbc168bd90073171102f3a22377c254b0f773693...,HÀ NỘI SẮP CÓ SHOW DIỄN THỰC CẢNH TRÊN SÔNG VE...,not-sarcasm,sarcasm
2723,2723,2723,00431202d34cca500b6fa9f7976fc3bd9f6bb98d525deb...,Áp lực công việc,multi-sarcasm,"Anh xem giup em, Xem gì em, Có phải em béo lên..."
4714,4714,4714,46827d1caad00dc07b80d1863c969d10f2da82620e1a61...,Trong cặp thằng nào có 10 tờ này thì giờ kiểm ...,multi-sarcasm,"Trường :, Thứ, ngày, tháng, năm, Lớp:, KIỂM TR..."
3041,3041,3041,5727101c2f08ff96e02f1de2cc92cdb1d5459c530a15ad...,Tây Du Ký phiên bản sắc nét 4K 😌,not-sarcasm,sarcasm
1779,1779,1779,c90b35765c434dfefddac499b96999a4ccfe92aa501bbf...,"Sáng nay, 6/6 hơn 96.000 học sinh tại TPHCM ch...",not-sarcasm,sarcasm


In [11]:
def text_to_emoji(text):
    emoji_map = {
        ":))": "😊",
        "=)))": "😊",# Happy
        ":((": "😞",  # Sad
        ":'(": "😢",  # Crying
        ":D": "😃",   # Big smile
        ":(": "☹️",   # Disappointed
        ":|": "😐",   # Neutral
    }
    for text_emoji, real_emoji in emoji_map.items():
        text = text.replace(text_emoji, real_emoji)
    return text

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text_to_emoji(text)
    text = re.sub(r'[^\w\s,]', '', text)
    text = re.sub(r'\d+', '', text)
    text = ' '.join(text.split())
    return text

df_split['caption']=df_split['caption'].apply(preprocess_text).apply(text_to_emoji)

In [12]:
df_split['caption']

0                       cô ấy trên mạng vs cô ấy ngoài đời
1               người tâm linh giao tiếp với người thực tế
2        hình như trăng hôm nay đẹp quá mọi người ạ canhco
3        mọi người nghĩ sao về phát biểu của shark việt...
4                tay hai nàng chứ việc gì phải lệ hai hàng
                               ...                        
10800                                          lộn đầu rồi
10801    chào các bạn, mình là goda takeshi trong live ...
10802                                   cre hùynh quốc huy
10803                                     anh hùng thật sự
10804                  quá là bình thường butchr interpool
Name: caption, Length: 10805, dtype: object

In [13]:
from sklearn.model_selection import train_test_split
# split train and dev
df_train, df_dev= train_test_split(df_split, test_size=0.1, random_state=42)

In [14]:
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np

# Assuming 'df' is your DataFrame and 'label' is the column with class labels

# Step 1: Calculate the sampling weight for each class
class_counts = df_train['label'].value_counts()
class_weights = 1.0 / class_counts
sample_weights = df_train['label'].map(class_weights).values  # Map the class weights to each sample

In [15]:
print(len(df_train))
print(len(df_dev))
print(len(df_test))

9724
1081
1413


In [16]:
IMAGE_TRAIN_FOLDER='/kaggle/input/dsc2024/training-images/train-images/'
IMAGE_TEST_FOLDER='/kaggle/input/dsc2024/private-test-images/test-images/'

In [17]:
#Encode labels
label_to_id = {lab:i for i, lab in enumerate(df_train['label'].sort_values().unique())}
id_to_label = {v:k for k,v in label_to_id.items()}
label_to_id

{'image-sarcasm': 0, 'multi-sarcasm': 1, 'not-sarcasm': 2, 'text-sarcasm': 3}

In [18]:
num_out_labels = len(label_to_id)
print("Number of labels ", num_out_labels)

Number of labels  4


In [19]:
# extract layers of resnet-50 to build a new model

import torch.nn as nn
from torchvision.models.resnet import resnet50

class ResNetFeatureModel(nn.Module):
    def __init__(self, output_layer):
        super().__init__()
        self.output_layer = output_layer 
        pretrained_resnet = resnet50(pretrained=True)
        self.children_list = []
        for n,c in pretrained_resnet.named_children():
            self.children_list.append(c)
            if n == self.output_layer:
                break

        self.net = nn.Sequential(*self.children_list)


    def forward(self,x):
        x = self.net(x)
        x = torch.flatten(x, 1)
        return x

In [20]:
import torch
import torch.nn as nn
from transformers import BeitModel, BeitConfig

class BeitFeatureModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Load pre-trained BEiT model
        self.beit = BeitModel.from_pretrained("microsoft/beit-base-patch16-224")

    def forward(self, x):
        # Extract features for the image
        outputs = self.beit(pixel_values=x)
        # Get the [CLS] token representation
        img_features = outputs.last_hidden_state[:, 0, :]  # Shape: (batch_size, hidden_size)
        return img_features


In [21]:
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class MultimodalDataset(Dataset):
    def __init__(self, df, label_to_id=None, mode='train', text_field="caption", label_field='label', image_path_field="image"):
        """
        Args:
            df (DataFrame): The DataFrame containing your data.
            label_to_id (dict): Dictionary for mapping labels to IDs. Set to None for test data.
            mode (str): Mode of the dataset. One of ['train', 'test', 'dev'].
            text_field (str): Column name for text data.
            label_field (str): Column name for label data.
            image_path_field (str): Column name for image paths.
        """
        self.df = df.reset_index(drop=True)
        self.label_to_id = label_to_id
        self.mode = mode  # Mode can be 'train', 'test', or 'dev'
        self.text_field = text_field
        #self.ocr=ocr_field
        self.label_field = label_field
        self.image_path_field = image_path_field

        # ResNet-50 settings
        self.img_size = 224
        self.mean, self.std = (
            0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)

        # Define different transformations based on the mode
        self.train_transform_func = transforms.Compose([
            transforms.RandomResizedCrop(self.img_size, scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        self.test_transform_func = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(self.img_size),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        self.dev_transform_func = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(self.img_size),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

    def __getitem__(self, index):
        # Get text data
        text = str(self.df.at[index, self.text_field])
        
        # Select the correct image folder based on mode
        if self.mode == 'test':
            img_path = IMAGE_TEST_FOLDER + self.df.at[index, self.image_path_field]
        else:
            img_path = IMAGE_TRAIN_FOLDER + self.df.at[index, self.image_path_field]

        # Load the image
        image = Image.open(img_path).convert('RGB')  # Ensure the image is in RGB format

        # Apply appropriate transformations based on mode
        if self.mode == 'train':
            img = self.train_transform_func(image)
        elif self.mode == 'test':
            img = self.test_transform_func(image)
        elif self.mode == 'dev':
            img = self.dev_transform_func(image)

        # If labels are available, return them, else only return the image and text
        if self.label_to_id is not None and self.label_field in self.df.columns:
            label = self.label_to_id[self.df.at[index, self.label_field]]
            return text, label, img
        else:
            return text, img

    def __len__(self):
        return self.df.shape[0]


In [22]:
class VisoBertBeitModel(nn.Module):
    def __init__(self, num_labels, text_pretrained='uitnlp/visobert', mlp_hidden_size=512, dropout_prob=0.3):
        super().__init__()
        # Text encoder (ViSoBERT)
        self.text_encoder = AutoModel.from_pretrained(text_pretrained)
        self.tokenizer = AutoTokenizer.from_pretrained(text_pretrained)

        # Visual encoder (BEiT)
        self.visual_encoder = BeitFeatureModel()
        
        # Check hidden sizes
        self.text_hidden_size = self.text_encoder.config.hidden_size
        self.image_hidden_size = self.visual_encoder.beit.config.hidden_size

        # MLP with one hidden layer
        self.mlp = nn.Sequential(
            nn.Linear(self.text_hidden_size + self.image_hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
        )

        # Classifier layer
        self.classifier = nn.Linear(mlp_hidden_size, num_labels)

    def forward(self, text, image):
        # Encode text and extract the [CLS] token feature
        text_output = self.text_encoder(**text)
        text_feature = text_output.last_hidden_state[:, 0, :]  # Shape: (batch_size, text_hidden_size)

        # Encode image features
        img_feature = self.visual_encoder(image)  # Shape: (batch_size, image_hidden_size)
        
        # Concatenate text and image features
        features = torch.cat((text_feature, img_feature), dim=1)  # Shape: (batch_size, text_hidden_size + image_hidden_size)

        # Pass through the MLP layer
        mlp_output = self.mlp(features)

        # Classify using the final output of the MLP
        logits = self.classifier(mlp_output)

        return logits

In [23]:
import torch
import numpy as np
from torch.utils.data import DataLoader, WeightedRandomSampler

def create_oversampled_dataloader(dataset, label_col, batch_size, num_workers=4):
    # Đếm số lượng mẫu của mỗi class trong dataset ban đầu
    class_sample_counts = dataset.df[label_col].value_counts().to_dict()
    
    # Tính trọng số cho mỗi sample dựa trên số lượng mẫu của class đó
    weights = 1. / np.array([class_sample_counts[label] for label in dataset.df[label_col]])
    
    # In thông tin về dataset trước khi oversampling
    print("Số lượng mẫu của mỗi class trước khi oversampling:", class_sample_counts)

    # Tạo WeightedRandomSampler với trọng số đã tính
    sampler = WeightedRandomSampler(weights, num_samples=len(dataset), replacement=True)

    # Tạo DataLoader với sampler
    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler, num_workers=num_workers)

    # Tính alpha trước khi oversampling
    total_count_before = sum(class_sample_counts.values())
    alpha_before_oversampling = {class_label: count / total_count_before for class_label, count in class_sample_counts.items()}
    print("Alpha trước khi oversampling:", alpha_before_oversampling)

    # Tính số lượng mẫu sau khi oversampling
    class_counts_after_sampling = {i: 0 for i in range(len(class_sample_counts))}
    
    for index in sampler:
        label = dataset[index][2]  # Index 2 là nhãn (label)
        class_counts_after_sampling[label] += 1
    
    # In thông tin về dataset sau khi oversampling
    print("Số lượng mẫu của mỗi class sau khi oversampling:", class_counts_after_sampling)

    # Tính alpha sau khi oversampling
    total_count_after = sum(class_counts_after_sampling.values())
    alpha_after_oversampling = {class_label: count / total_count_after for class_label, count in class_counts_after_sampling.items()}
    print("Alpha sau khi oversampling:", alpha_after_oversampling)

    # Chuyển alpha thành tensor để sử dụng trong Focal Loss
    alpha_for_focal = torch.tensor([alpha_after_oversampling[i] for i in range(len(alpha_after_oversampling))], dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')

    return dataloader, alpha_for_focal


In [24]:
len(label_to_id)

4

In [25]:
import torch

# Specify the path to the saved model
model_path = "/kaggle/input/dsc2024/resnet_model_v8.pth"

# Assuming the same model architecture is defined
model = VisoBertBeitModel(num_labels=num_out_labels, text_pretrained='uitnlp/visobert')
#model = VisoBertResNetModel(num_labels=len(label_to_id), text_pretrained='vinai/phobert-base')

 # Replace `MyModel` with your actual model class

# Load the state_dict from the file
model.load_state_dict(torch.load(model_path))

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)

# Set the model to evaluation mode
#model.train()

#print(f"Model loaded from {model_path}")


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/390M [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at uitnlp/visobert and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


sentencepiece.bpe.model:   0%|          | 0.00/471k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/69.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/350M [00:00<?, ?B/s]

/tmp/ipykernel_30/849942651.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


In [26]:
len(df_train)

9724

In [27]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2d2aefef-4c29-4559-6c8f-6794e2121533)
GPU 1: Tesla T4 (UUID: GPU-7e685d48-1d93-5c8a-6320-84cb3f574da4)


In [28]:
# parameters
training_params = {
    "seed_val": seed_val,
    "training_size" : len(df_train),
    "dev_size": len(df_dev),
    "test_size": len(df_test),
    "num_train_epochs": 30,
    "batch_size": 16,
    "learning_rate": 1e-5, 
    "weight_decay": 0.01,
    "warmup_steps": 0,
    "max_seq_length": 512
}


In [29]:
# Accessing each value by key
seed_val = training_params['seed_val']
training_size = training_params['training_size']
dev_size = training_params['dev_size']
test_size = training_params['test_size']
num_train_epochs = training_params['num_train_epochs']
batch_size = training_params['batch_size']
learning_rate = training_params['learning_rate']
weight_decay = training_params['weight_decay']
warmup_steps = training_params['warmup_steps']
max_seq_length = training_params['max_seq_length']


In [30]:
training_params

{'seed_val': 0,
 'training_size': 9724,
 'dev_size': 1081,
 'test_size': 1413,
 'num_train_epochs': 10,
 'batch_size': 32,
 'learning_rate': 1e-05,
 'weight_decay': 0.01,
 'warmup_steps': 0,
 'max_seq_length': 512}

In [31]:


# Step 2: Create the WeightedRandomSampler
#sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)



In [ ]:
# training step
import matplotlib.pyplot as plt
import torch
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, RandomSampler
from tqdm import trange, tqdm
import torch.nn as nn
from transformers import get_scheduler, AdamW
import time


# Set up gradient accumulation steps and use mixed precision
accumulation_steps = 4  # Perform backward pass and optimizer step after this many batches
scaler = GradScaler()   # For mixed precision

train_dataset = MultimodalDataset(df=df_train, label_to_id=label_to_id, mode='train', text_field='caption', label_field='label', image_path_field='image')
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=train_sampler)
#train_dataloader, class_weights = create_oversampled_dataloader(train_dataset, label_col='label', batch_size=batch_size)  # Tăng num_workers để tải dữ liệu song song

# Step 3: Use the sampler in the DataLoader
#train_dataloader = DataLoader(
 #   df_train,        # Replace with your dataset object
  #  batch_size=batch_size,       # Set batch size as desired
   # sampler=sampler      # Pass the sampler
#)
t_total = len(train_dataloader) * num_train_epochs

optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = get_scheduler(name="cosine", optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

# Set the device to the second GPU (GPU 1)
# Move model to the device
criterion = nn.CrossEntropyLoss()
#criterion = L.CrossEntropyFocalLoss(gamma=1.0, reduction='mean', class_weights=class_weights)

start = time.perf_counter()

#for param in model.text_encoder.parameters():
 #   param.requires_grad = True

#for param in model.visual_encoder.parameters():
 #   param.requires_grad = True


# Initialize lists to store epoch loss and learning rate
epoch_losses = []
learning_rates = []

# Set up training loop with tqdm only for epochs
for epoch_num in trange(num_train_epochs, desc='Epochs'):
    model.train()  # Set the model to training mode
    epoch_total_loss = 0
    
    for step, batch in enumerate(train_dataloader):
        b_text, b_labels, b_imgs = batch
        
        # Tokenize text input
        b_inputs = model.tokenizer(
            list(b_text), truncation=True, max_length=max_seq_length,
            return_tensors="pt", padding=True
        )
        
        # Move labels, images, and inputs to the GPU
        b_labels = b_labels.to(device)
        b_imgs = b_imgs.to(device)
        b_inputs = {k: v.to(device) for k, v in b_inputs.items()}

        # Enable mixed precision using autocast
        with autocast():
            b_logits = model(text=b_inputs, image=b_imgs)  # Forward pass
            loss = criterion(b_logits, b_labels)  # Calculate loss
        
        # Accumulate loss for gradient accumulation
        loss = loss / accumulation_steps
        scaler.scale(loss).backward()  # Scale and backpropagate the loss
        
        # Perform optimizer step after the defined accumulation steps
        if (step + 1) % accumulation_steps == 0 or (step + 1) == len(train_dataloader):
            scaler.step(optimizer)  # Perform the optimizer step
            scaler.update()  # Update the scale for next iteration
            optimizer.zero_grad()  # Clear gradients
            scheduler.step()  # Update learning rate at the end of each batch
        
        # Accumulate the total loss
        epoch_total_loss += loss.item() * accumulation_steps

    # Compute average loss for the epoch
    avg_loss = epoch_total_loss / len(train_dataloader)

    # Save the average loss and learning rate for this epoch
    epoch_losses.append(avg_loss)
    learning_rates.append(optimizer.param_groups[0]['lr'])
    torch.cuda.empty_cache()  # Clear unused cached memory after each epoch


    # Print results after each epoch
    print(f'Epoch = {epoch_num + 1}')
    print(f'    Epoch loss = {epoch_total_loss}')
    print(f'    Average epoch loss = {avg_loss}')
    print(f'    Learning rate = {optimizer.param_groups[0]["lr"]}')

end = time.perf_counter()
resnet_training_time = end - start
print(f'Training completed in {resnet_training_time} seconds')

# Plot the loss and learning rate curves
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(range(1, num_train_epochs + 1), epoch_losses, label="Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss per Epoch")
plt.grid(True)

# Plot learning rate
plt.subplot(1, 2, 2)
plt.plot(range(1, num_train_epochs + 1), learning_rates, label="Learning Rate", color='orange')
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.title("Learning Rate per Epoch")
plt.grid(True)




/tmp/ipykernel_30/3477076540.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()   # For mixed precision
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epochs:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_30/3477076540.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epochs:  10%|█         | 1/10 [07:19<1:05:53, 439.27s/it]

Epoch = 1
    Epoch loss = 33.54843958187848
    Average epoch loss = 0.11035670915091607
    Learning rate = 9.984586668665641e-06


Epochs:  20%|██        | 2/10 [13:32<53:23, 400.45s/it]  

Epoch = 2
    Epoch loss = 28.583597861230373
    Average epoch loss = 0.0940249929645736
    Learning rate = 9.938441702975689e-06


Epochs:  30%|███       | 3/10 [19:50<45:29, 389.96s/it]

...

Epochs:  60%|██████    | 6/10 [38:34<25:14, 378.62s/it]

Epoch = 6
    Epoch loss = 18.188053887337446
    Average epoch loss = 0.059829124629399495
    Learning rate = 9.45503262094184e-06


Epochs:  70%|███████   | 7/10 [44:51<18:54, 378.09s/it]

Epoch = 7
    Epoch loss = 17.186412513256073
    Average epoch loss = 0.05653425168834234
    Learning rate = 9.263200821770462e-06


Epochs:  80%|████████  | 8/10 [51:09<12:36, 378.09s/it]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#save model
import torch

# Assuming your trained model is called 'model'
import os
# Path to save the model
model_save_path = 'beit_model.pth'

# Save the model's state_dict
torch.save(model.state_dict(), model_save_path)

print(f"Model saved to {model_save_path}")


In [ ]:
# testing loop

resnet_prediction_results = []

dev_dataset = MultimodalDataset(df=df_dev, label_to_id=label_to_id, mode='dev', text_field='caption', label_field='label', image_path_field='image')
test_sampler = SequentialSampler(dev_dataset)
test_dataloader = DataLoader(dataset=dev_dataset,
                            batch_size=batch_size,
                            sampler=test_sampler)


for batch in tqdm(test_dataloader):

  b_text, b_labels, b_imgs = batch

  b_inputs = model.tokenizer(list(b_text), truncation=True, max_length=max_seq_length, return_tensors="pt", padding=True)

  b_labels = b_labels.to(device)
  b_imgs = b_imgs.to(device)
  b_inputs = b_inputs.to(device)
  model.eval()
  with torch.no_grad():
      b_logits = model(text=b_inputs, image=b_imgs)
      b_logits = b_logits.detach().cpu()

  resnet_prediction_results += torch.argmax(b_logits, dim=-1).tolist()

resnet_prediction_labels = [id_to_label[p] for p in resnet_prediction_results]

In [ ]:
len(resnet_prediction_labels)

In [ ]:
if (len(resnet_prediction_labels) == len(df_dev)):
    print(True)

In [ ]:
resnet_class_report = classification_report(df_dev['label'], resnet_prediction_labels)



In [ ]:
resnet_class_report

In [ ]:
training_params['results']=resnet_class_report

In [ ]:
training_params

In [ ]:
# Specify the file path
file_path = "/kaggle/working/training_report.txt"

# Format the dictionary as a string
report_content = "\n".join([f"{key}: {value}" for key, value in training_params.items()])

# Save the report to a text file
with open(file_path, mode="w") as file:
    file.write(report_content)

print(f"Report saved to {file_path}")


# Making prediction


In [ ]:
df_test

In [ ]:
# Assuming df_test has columns ['caption', 'image']

# Initialize the dataset for the test set (no labels, test transformation)
test_dataset = MultimodalDataset(
    df=df_test,  # DataFrame containing the new data
    label_to_id=None,  # No labels for prediction
    mode='test',  # Since it's a test dataset, set train=False for evaluation transforms
    text_field="caption", # Column for captions
    image_path_field="image"  # Column for image paths
)

# Set up a DataLoader for the test dataset
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,  # Adjust your batch size as needed
    sampler=test_sampler
)

# Now you can use the test_dataloader in a prediction loop
prediction_results = []
# Set the model to evaluation mode

for batch in tqdm(test_dataloader):
    b_text, b_imgs = batch  # No labels are unpacked here

    # Tokenize the input text (captions)
    b_inputs = model.tokenizer(
        list(b_text), truncation=True, max_length=max_seq_length, return_tensors="pt", padding=True
    )

    # Move inputs to the correct device
    b_imgs = b_imgs.to(device)
    b_inputs = {k: v.to(device) for k, v in b_inputs.items()}

    # Perform inference
    model.eval() 
    with torch.no_grad():
        b_logits = model(text=b_inputs, image=b_imgs)
        b_logits = b_logits.detach().cpu()  # Move logits to CPU for further processing

    # Collect predictions
    prediction_results += torch.argmax(b_logits, dim=-1).tolist()

# If you have an id_to_label mapping, convert indices to labels (if needed)
if 'id_to_label' in locals():
    predicted_labels = [id_to_label[p] for p in prediction_results]
else:
    predicted_labels = prediction_results  # Return indices if no label mapping exists

# Output predictions


In [ ]:
df_test.sample(5)

In [ ]:
if(len(predicted_labels)==len(df_test)):
    print(True)
else: print(False)

In [ ]:
# Create the submission dictionary using index from iterrows
prediction_results = {
    "results": {
        str(idx): predicted_labels[i] for i, (idx, row) in enumerate(df_test.iterrows())  # Map index to predicted labels
    },
    "phase": "dev"  # Set phase as 'test', 'dev', or 'train' as appropriate
}

# Print the structured results
print(prediction_results)


In [ ]:
import json

# File path where you want to save the JSON file
output_file = "results.json"

# Save the dictionary as a JSON file
with open(output_file, 'w') as f:
    json.dump(prediction_results, f, indent=4)

print(f"Prediction results saved to {output_file}")
